In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import boto3
import re
from datetime import datetime

import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig
from sagemaker.model_monitor import DataCaptureConfig, DatasetFormat, DefaultModelMonitor
from sagemaker.s3 import S3Uploader, S3Downloader

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

%cd /root/predicting-coronavirus
#%pip install mpu
import src.data_import as di
import src.data_tools as dt

%matplotlib inline
pd.set_option('display.max_columns', 500)  
pd.set_option('display.max_rows', 500)   

/root/predicting-coronavirus


In [4]:
sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()

In [5]:
account_id = sess.client('sts', region_name=sess.region_name).get_caller_identity()["Account"]
bucket = 'sagemaker-studio-{}-{}'.format(sess.region_name, account_id)
prefix = 'capstone2'

try:
    if sess.region_name == "us-east-1":
        sess.client('s3').create_bucket(Bucket=bucket)
    else:
        sess.client('s3').create_bucket(Bucket=bucket, 
                                        CreateBucketConfiguration={'LocationConstraint': sess.region_name})
except Exception as e:
    print("Looks like you already have a bucket of this name. That's good. Uploading the data files...")

# Return the URLs of the uploaded file, so they can be reviewed or used elsewhere
s3url = S3Uploader.upload('data/train3k.csv', 's3://{}/{}/{}'.format(bucket, prefix,'merged'))
s3url = S3Uploader.upload('data/val3k.csv', 's3://{}/{}/{}'.format(bucket, prefix,'merged'))
print(s3url)


s3://sagemaker-studio-us-east-1-752222400982/capstone2/merged/val3k.csv


In [6]:
from sagemaker.amazon.amazon_estimator import get_image_uri
docker_image_name = get_image_uri(boto3.Session().region_name, 'xgboost', repo_version='1.0-1')

In [10]:
s3_input_train = sagemaker.s3_input(s3_data='s3://sagemaker-studio-us-east-1-752222400982/capstone2/merged/train2.csv', content_type='csv')
s3_input_val = sagemaker.s3_input(s3_data='s3://sagemaker-studio-us-east-1-752222400982/capstone2/merged/val2.csv', content_type='csv')


In [11]:
smsess = sagemaker.session.Session()

create_date = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
coronavirus_xgb_experiment = Experiment.create(experiment_name="capstone2-xgb-{}".format(create_date), 
                                              description="Using xgboost to predict coronavirus cases", 
                                              sagemaker_boto_client=boto3.client('sagemaker'))

In [21]:
from sagemaker.tuner import IntegerParameter, ContinuousParameter, HyperparameterTuner

trial = Trial.create(trial_name="algorithm-mode-trial-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime())), 
                     experiment_name=coronavirus_xgb_experiment.experiment_name,
                     sagemaker_boto_client=boto3.client('sagemaker'))

xgb = sagemaker.estimator.Estimator(image_name=docker_image_name,
                                    role=role,
                                    train_instance_count=1, 
                                    train_use_spot_instances=True,
                                    train_max_run=300,
                                    train_max_wait=600,
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    base_job_name="xgboost-coronavirus",
                                    sagemaker_session=smsess)

xgb.set_hyperparameters(num_round=30, 
                        objective='reg:squarederror',
                        verbosity=1,
                        eta=0.1,
                        subsample=0.78,
                        min_child_weight=3,
                        gamma=4)

hp_ranges = {
    "max_depth":IntegerParameter(10,20)
#   "subsample":ContinuousParameter(0.4, 0.8),
#    "eta":ContinuousParameter(0.03,0.2),
#   "min_child_weight":IntegerParameter(1, 8),
#    "alpha":ContinuousParameter(0, 2)
}

tuner = HyperparameterTuner(
    xgb,
    'validation:rmse',
    hp_ranges,
    objective_type='Minimize',
    max_jobs=10,
    max_parallel_jobs=10
)

tuner.fit({'train': s3_input_train,
         'validation': s3_input_val}, 
                experiment_config={
                    "ExperimentName": coronavirus_xgb_experiment.experiment_name, 
                    "TrialName": trial.trial_name,
                    "TrialComponentDisplayName": "HPO Job"
                })



INFO:root:_TuningJob.start_new!!!
INFO:sagemaker:Creating hyperparameter tuning job with name: sagemaker-xgboost-200515-1356


In [23]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

exp = HyperparameterTuningJobAnalytics(
    sagemaker_session=smsess,
    hyperparameter_tuning_job_name=tuner.latest_tuning_job.name)
df = exp.dataframe()
df.sort_values(by='FinalObjectiveValue')

,max_depth,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
9,15.0,sagemaker-xgboost-200515-1356-001-b0a7b39b,Completed,203.370392,2020-05-15 14:00:06+00:00,2020-05-15 14:01:10+00:00,64.0
7,19.0,sagemaker-xgboost-200515-1356-003-81db6580,Completed,203.378983,2020-05-15 13:59:50+00:00,2020-05-15 14:01:07+00:00,77.0
0,18.0,sagemaker-xgboost-200515-1356-010-5c6f33c6,Completed,203.381271,2020-05-15 13:59:59+00:00,2020-05-15 14:01:15+00:00,76.0
5,20.0,sagemaker-xgboost-200515-1356-005-cf83641e,Completed,203.381271,2020-05-15 13:59:38+00:00,2020-05-15 14:00:44+00:00,66.0
3,14.0,sagemaker-xgboost-200515-1356-007-6a2e76b8,Completed,203.386902,2020-05-15 13:59:37+00:00,2020-05-15 14:00:50+00:00,73.0
6,17.0,sagemaker-xgboost-200515-1356-004-8e5c5f54,Completed,203.387772,2020-05-15 13:59:24+00:00,2020-05-15 14:00:23+00:00,59.0
8,13.0,sagemaker-xgboost-200515-1356-002-51a9cfbe,Completed,203.407150,2020-05-15 13:59:53+00:00,2020-05-15 14:01:09+00:00,76.0
4,12.0,sagemaker-xgboost-200515-1356-006-35237f01,Completed,203.440201,2020-05-15 13:59:57+00:00,2020-05-15 14:00:58+00:00,61.0
1,11.0,sagemaker-xgboost-200515-1356-009-64d3eaa9,Completed,203.552826,2020-05-15 13:59:42+00:00,2020-05-15 14:00:43+00:00,61.0
2,10.0,sagemaker-xgboost-200515-1356-008-c0d3f079,Completed,203.728607,2020-05-15 13:59:26+00:00,2020-05-15 14:00:38+00:00,72.0
